In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set(style='darkgrid')

In [ ]:
# import csv as DF
df = pd.read_csv('../data/dataframe.csv')

In [ ]:
df

In [ ]:
df['walkscore_category'] = ''

def ws_cat(df):
    str_cat = ''
    if df['walkscore'] >= 90: 
        str_cat = "Walker's Paradise"
    elif 90 > df['walkscore'] >= 70: 
        str_cat = "Very Walkable"
    elif 70 > df['walkscore'] >= 50: 
        str_cat = "Somewhat Walkable"
    elif 50 > df['walkscore'] >= 25:
        str_cat = "Car-Dependent_2"
    elif 25 > df['walkscore'] >= 0:
        str_cat = "Car-Dependent_1"
    return str_cat

def ws_cat_num(df):
    num_cat = 0
    if df['walkscore'] >= 90: 
        num_cat = 5
    elif 90 > df['walkscore'] >= 70: 
        num_cat = 4
    elif 70 > df['walkscore'] >= 50: 
        num_cat = 3
    elif 50 > df['walkscore'] >= 25:
        num_cat = 2
    elif 25 > df['walkscore'] >= 0:
        num_cat = 1
    return num_cat

df['walkscore_category'] = df.apply(ws_cat, axis=1)
df['walkscore_category_num'] = df.apply(ws_cat_num, axis=1)

df

In [ ]:
# remove outliers
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1
print(IQR)

In [ ]:
z_cost_sf = np.abs(stats.zscore(df['cost/SF']))
print(z_cost_sf)

In [ ]:
threshold = 3
outliers_cost_sf = np.where(z_cost_sf > threshold)
print(outliers_cost_sf)

In [ ]:
df_result = df.copy()
for apt in outliers_cost_sf:
    df_result.drop(apt, inplace=True)
#     outliers.append(df.iloc[int(apt)])
#     print(outliers)
df_result

# dropped 7 outliers

In [ ]:
df_outliers = df.iloc[outliers_cost_sf]
df_outliers

In [ ]:
df = df_result.copy()
df

In [ ]:
y = df['mean_rent']
sns.distplot(df['mean_rent'], bins=50, kde=False);

In [ ]:
# scatter plot showing no correlation betwee rent and walkscore
sns.scatterplot(df['mean_rent'], df['walkscore'], alpha=.5)

In [ ]:
# scatter plot maybe showing very weak negative correlation between unit area and walkscore
sns.scatterplot(df['mean_area'], df['walkscore'], alpha=.5)

In [ ]:
sns.scatterplot(df['cost/SF'], df['walkscore'], alpha=.5);

In [ ]:
df['walkscore'] = pd.to_numeric(df['walkscore'])

In [ ]:
type(df['walkscore'][0])

In [ ]:
df_walkscore = df[['mean_rent', 'mean_area', 'cost/SF', 'walkscore', 'walkscore_category', 'walkscore_category_num']]
df_walkscore = df_walkscore.sort_values(by='walkscore_category')

In [ ]:
df_walkscore

In [ ]:
ws5_cost_sf = df[df['walkscore_category'] == "Walker's Paradise"]['cost/SF'] 
ws4_cost_sf = df[df['walkscore_category'] == "Very Walkable"]['cost/SF']
ws3_cost_sf = df[df['walkscore_category'] == "Somewhat Walkable"]['cost/SF']
ws2_cost_sf = df[df['walkscore_category'] == "Car-Dependent_2"]['cost/SF']
ws1_cost_sf = df[df['walkscore_category'] == "Car-Dependent_1"]['cost/SF']

In [ ]:
# find group means
group_labels = ["Walker's Paradise", "Very Walkable", "Somewhat Walkable", "Car-Dependent_2", "Car-Dependent_1"]
groups_data = [ws5_cost_sf, ws4_cost_sf, ws3_cost_sf, ws2_cost_sf, ws1_cost_sf]
ws_dicts = {}

for label, data in zip(group_labels, groups_data):
    ws_dicts[label] = data
    print('mean for ', label, ': ', np.mean(ws_dicts[label]))

In [ ]:
# find group sizes
for label, data in ws_dicts.items():
    print('count for ', label, ': ', len(ws_dicts[label]))

In [ ]:
# '3d' scatter plot
plt.figure(figsize=(10, 10))
sns.scatterplot(x='mean_rent', y='mean_area', data=df_walkscore, hue='walkscore_category');

In [ ]:
# box plots
plt.figure(figsize=(10, 5))
sns.boxplot(x='cost/SF', 
            y='walkscore_category', 
            data=df_walkscore, 
            orient='h', 
            fliersize=3, 
            palette='colorblind', 
            showmeans=True, 
            meanprops={"marker":"o","markerfacecolor":"white", "markeredgecolor": "black"});

In [ ]:
# stats.f_oneway(ws5_cost_sf, ws4_cost_sf, ws3_cost_sf, ws2_cost_sf, ws1_cost_sf)
# stats.f_oneway(ws5_cost_sf, ws4_cost_sf.append([ws3_cost_sf, ws2_cost_sf, ws1_cost_sf]))

stats.f_oneway(ws5_cost_sf.append(ws4_cost_sf), ws3_cost_sf.append([ws2_cost_sf, ws1_cost_sf]))

# stats.f_oneway(ws5_cost_sf.append([ws4_cost_sf, ws3_cost_sf]), ws2_cost_sf.append(ws1_cost_sf))
# stats.f_oneway(ws5_cost_sf.append([ws4_cost_sf, ws3_cost_sf, ws2_cost_sf]), ws1_cost_sf)

In [ ]:
stats.ttest_ind(ws5_cost_sf.append(ws4_cost_sf), ws3_cost_sf.append([ws2_cost_sf, ws1_cost_sf]))

## p-value is much less than the significance level of 5%. We reject the null hypothesis. 